# Thuật toán lọc cộng tác kết hợp phân cụm k-means (K-Means Clustering CF)

Trên thực tế, tập dữ liệu là nhiều chiều và thưa thớt, để tiết kiệm thời gian cũng như giảm bớt các phép tính toán thì trước khi tiến hành dự đoán rating của user $u$ cho item $t$, ta tiến hành chia nhỏ tập dữ liệu ban đầu thành $k$ tập dữ con bằng thuật toán phân cụm k-means.

Thuật toán này gồm 2 giai đoạn chính:
1. Phân cụm tập dữ liệu ban đầu thành $k$ tập con, có thể phân cụm theo user hoặc phân cụm theo item.
2. Áp dụng thuật toán lọc cộng tác tương ứng với cách phân cụm để dự đoán rating của user mục tiêu với các item mà user này chưa đánh giá.

**Chi tiết hơn:**

1. Thuật toán K-means:
- Bước 1: Chọn $k$ điểm dữ liệu ngẫu nhiên làm centroid cho các cụm.
Tìm số cụm $k$ bằng phương pháp khuỷu tay (elbow method) bằng hàm biến dạng:

$$J(c, \mu) = \sum_{i=1}^n \|x^{(i)} - \mu_{c^{(i)}}\|^2$$

- Bước 2: Chạy vòng lặp sau:
  - Bước 2.1: Với mỗi điểm dữ liệu, tính khoảng cách giữa nó và $k$ centroid, và gán điểm dữ liệu này cho centroid có khoảng cách từ nó đến centroid đó là nhỏ nhất.
  - Bước 2.2: Khi tất cả các điểm dữ liệu được gán cho các cụm, đối với từng cụm, tính tổng khoảng cách giữa các item và centroid.
    - Nếu tổng khoảng cách không nhỏ hơn tổng khoảng cách của vòng lặp trước đó thì trả về các cụm
  - Bước 2.3: Cập nhật centroid là điểm trung bình của cụm

2. Thuật toán lọc cộng tác (xem lại bài thực hành 1 và 2)

Trong bài thực hành này, ta sẽ kết hợp K-means với lọc cộng tác dựa trên item, nghĩa là các item sẽ được gom nhóm lại nhau bằng thuật toán k-means và xét trường hợp đơn giản là coi tất cả các item trong cụm chứa item t là các item lân cận với item t.

### Ví dụ 1

In [1]:
# import libraries
import numpy as np
import pandas as pd
from functools import reduce
# hàm reduce là hàm gồm 2 đối số,
# đối số 1 là 1 hàm, đối số 2 là các giá trị sẽ truyền vào hàm ở đối số 1
# và kết quả sẽ tích luỹ dần
# Ví dụ: reduce(lambda x, y: x+y, [1, 2, 3, 4, 5])
# ((((1+2)+3)+4)+5) = 15


In [2]:
mx = np.array([[5,6,7,4,3,np.nan],
              [4,np.nan,3,np.nan,5,4],
              [np.nan,3,4,1,1,np.nan],
              [7,4,3,6,np.nan,4],
              [1,np.nan,3,2,2,5]])
df = pd.DataFrame(mx, index = ['u1', 'u2', 'u3', 'u4', 'u5'], columns = ['a', 'b', 'c', 'd', 'e', 'f'])
df

,a,b,c,d,e,f
u1,5.0,6.0,7.0,4.0,3.0,NaN
u2,4.0,NaN,3.0,NaN,5.0,4.0
u3,NaN,3.0,4.0,1.0,1.0,NaN
u4,7.0,4.0,3.0,6.0,NaN,4.0
u5,1.0,NaN,3.0,2.0,2.0,5.0


#### 1. Thuật toán K-means

In [3]:
def euclidean_dist(x,y):
  """Hàm tính khoảng cách Euclidean giữa hai vector x và y"""
  dist = 0
  mutual_index = [i for i in range(len(x)) if not np.isnan(x[i]) and not np.isnan(y[i])]
  if len(mutual_index) != 0:
    for i in mutual_index:
      dist += (x[i] - y[i])**2
    return (np.sqrt(dist)/len(mutual_index))
  else:
    return 0

def cosine(item_i, item_j):
  # Tìm index mà item_i và item_j đều được rate bởi tất cả các user
  mutual_index = [u for u in item_i.index if not np.isnan(item_i[u]) and not np.isnan(item_j[u])]

  dividend = 0
  i_divisor = 0
  j_divisor = 0

  # Tính độ tương đồng cosine
  for index in mutual_index:
    dividend += item_i[index] * item_j[index]
    i_divisor += pow(item_i[index], 2)
    j_divisor += pow(item_j[index], 2)

  divisor = np.sqrt(i_divisor) * np.sqrt(j_divisor) # Mẫu số của cosine

  if divisor != 0:
    return round(dividend / divisor, 3) # trả về độ tương đồng nếu mẫu số != 0

  return 0 # trả về 0 nếu mẫu số = 0

def add_vector(i, j):
  """Hàm cộng hai vector i và j"""
  return [np.nansum([i[k],j[k]]) for k in range(len(j))]

In [1]:
?min

In [4]:
def generate_centroids(k, data):
  """Chọn k items ngẫu nhiên của tập data làm centroids"""
  return data.sample(k, axis=1)

def assign_cluster(item, k, centroids):
  """Gán item vào cụm mà khoảng cách của item đến centroid của cụm đó có khoảng
  cách nhỏ nhất so với khoảng cách của item đến các centroids của các cụm khác
  @params:
    item: một item trong tập dữ liệu
    centroids: tập các centroids của các cụm
  @return:
    trả về cụm của item đó"""
  return min(range(k), key= lambda i: euclidean_dist(item.tolist(), centroids.iloc[:,i].tolist()))

def k_means(data, k):
  """
    Hàm nhóm các item thành k cụm
  """
  max_iter = 5
  iter = 0

  best_dist = np.inf # khởi tại khoảng cách ban đầu là vô định
  centroids = generate_centroids(k, data) # khởi tạo k centroids ngẫu nhiên từ tập data

  # Khởi tạo dict để lưu item và cụm tương ứng
  clusters = {}
  while iter < max_iter:
    # gán item vào cụm thứ k dựa trên khoảng cách nhỏ nhất giữa item đến các trung tâm cụm
    for item in data.columns:
      clusters[item] = assign_cluster(data[item], k, centroids)
    # Khởi tạo biến new_dist để lưu tổng khoảng cách mới
    new_dist = 0
    # Lặp qua từng assign_cluster(item: tên cụm tương ứng)
    for key, value in clusters.items():
      # Tính tổng khoảng cách của các item trong cụm đến centroid của cụm
      new_dist += euclidean_dist(data[key].tolist(), centroids.iloc[:,value].tolist())
    # Nếu tổng khoảng cách mới nhỏ hơn tổng khoảng cách của vòng lặp trước
    # thì cập nhật dist
    if new_dist < best_dist:
      best_dist = new_dist
      new_dist = 0
    else: # ngược lại trả về clusters: {item: cụm tương ứng} và centroids
      return clusters,  centroids
    # Cập nhật lại centroid của các cụm
    centroids = update_centroids(data, k, clusters)
    iter += 1

def update_centroids(data, k, clusters):
  """
  Hàm cập nhật lại centroid của các cụm
  """
  centroids = {}
  # Lặp qua k cụm để tạo các centroids mới
  for cen in range(k):
    # Tìm tất cả các thành viên của cụm này
    members = [data[key].tolist() for key, value in clusters.items() if value == cen]
    if members:
      l = [i/len(members) for i in reduce(add_vector, members)]
      centroids[cen] = l
  return pd.DataFrame.from_dict(centroids)

In [5]:
k = 2
item_cluster, centroids = k_means(df, k)

item_cluster_df = pd.DataFrame(item_cluster.items(), columns=['item', 'cluster'])
display(item_cluster_df)

for i in range(k):
  len_items = item_cluster_df[item_cluster_df['cluster'] == i].shape[0]
  print(f'Số lượng item trong cluster {i}: {len_items}')

,item,cluster
0,a,1
1,b,0
2,c,0
3,d,1
4,e,1
5,f,0


Số lượng item trong cluster 0: 3
Số lượng item trong cluster 1: 3


#### 2. Lọc cộng tác dựa trên item

In [6]:
def clustered_item_data(data, item_cluster, target_item):
  """Hàm trả về dataframe của cụm chứa target_item
  @params:
    data: data ban đầu
    item_cluster: dataframe gồm 2 cột: item và cụm tương ứng
    target_item: item mục tiêu
  @return: dataframe rating của user với các item của cụm có chứa target_item"""
  # Lấy tên cụm của target_item
  c = item_cluster[item_cluster['item'] == target_item]['cluster'].values
  # lấy tên tất cả các item trong cụm c
  item_list = list(item_cluster[item_cluster['cluster'] == c[0]]['item'])
  # Truy xuất lại dữ liệu ban đầu chỉ gồm những item thuộc về cụm c
  return data[item_list]

In [9]:
# Hàm dự đoán số điểm rating của user mục tiêu cho item t
# Bài toán ở đây xét trường hợp đơn giản là coi tất cả các item trong cụm chứa item t
# là các item lân cận với item t
def item_based_rec(data, item_cluster, target_user, threshold=0.2):
  """@param:
    data: dataframe user-item ban đầu
    target_user: user mục tiêu
    item_cluster: dataframe gồm 2 cột: item và cụm tương ứng
    @return: số điểm rating của target_user cho các item mà target_user chưa đánh giá
  """

  # Những item mà user mục tiêu đã đánh giá
  target_user_rated = data[data.index==target_user].dropna(axis=1)
  target_user_rated_df = target_user_rated.T.reset_index().rename(columns={target_user:'rating', 'index':'item'}) # chuyển vị, reset lại index và đổi tên cột

  # Những item mà user mục tiêu chưa đánh giá
  target_user_unrated = data.drop(target_user_rated.columns, axis=1, errors='ignore')

  #print(f"Các item mà user mục tiêu chưa đánh giá: {target_user_unrated.columns.values}")

  # Khởi tạo dictionary to lưu những item chưa được đánh giá và số điểm dự đoán tương ứng
  rating_prediction ={}

  # Lặp qua từng item trong tập item chưa được đánh giá
  for picked_item in target_user_unrated:
    # Truy xuất cụm data có chứa picked_item
    clustered_data = clustered_item_data(data, item_cluster, picked_item)

    #print(f"\n Cụm dữ liệu có chứa {picked_item} :")
    #display(clustered_data)

    # Chuẩn hóa dữ liệu
    normalized_df = clustered_data.subtract(clustered_data.mean(axis=1), axis='rows')

    # Khởi tạo tử số và mẫu số cho dự đoán rating
    nominator = 0
    denominator = 0
    # Chạy qua từng item trong cụm data
    for item in clustered_data:
      if item != picked_item:
        # Tính hệ số cosine hiệu chỉnh giữa picked_item và từng item
        similarity= cosine(normalized_df[picked_item], normalized_df[item])

        #print(f"Cosine similarity ({picked_item}, {item}): {similarity}")
        rating = clustered_data[clustered_data.index==target_user][item].values[0]
        if (pd.isna(rating) == False)& (similarity > threshold):
          nominator += rating * similarity
          denominator += similarity
    if(denominator != 0):
      rating_prediction[picked_item] = nominator/denominator
    else:
      rating_prediction[picked_item] = 0

  # Bước 7: Chuyển dictionary thành dataframe và sắp xếp dữ liệu theo prediction_score
  pred_score = (pd.DataFrame(rating_prediction.items(), columns=['item', 'pred_score'])
                      .sort_values(by='pred_score', ascending=False))
    # Trả về dataframe gồm item mà target_user chưa đánh giá cùng với số điểm rating dự đoán
  return pred_score

In [8]:
target_user = 'u2'

k = 2
item_cluster, centroids = k_means(df, k)

item_cluster_df = pd.DataFrame(item_cluster.items(), columns=['item', 'cluster'])

for i in range(k):
  len_items = item_cluster_df[item_cluster_df['cluster'] == i].shape[0]
  print(f'Số lượng item trong cluster {i}: {len_items}')

display(df)
display(item_cluster_df)
item_based_rec(df, item_cluster_df, target_user)

Số lượng item trong cluster 0: 3
Số lượng item trong cluster 1: 3


,a,b,c,d,e,f
u1,5.0,6.0,7.0,4.0,3.0,NaN
u2,4.0,NaN,3.0,NaN,5.0,4.0
u3,NaN,3.0,4.0,1.0,1.0,NaN
u4,7.0,4.0,3.0,6.0,NaN,4.0
u5,1.0,NaN,3.0,2.0,2.0,5.0


,item,cluster
0,a,0
1,b,1
2,c,1
3,d,0
4,e,0
5,f,1


Các item mà user mục tiêu chưa đánh giá: ['b' 'd']

 Cụm dữ liệu có chứa b :


,b,c,f
u1,6.0,7.0,NaN
u2,NaN,3.0,4.0
u3,3.0,4.0,NaN
u4,4.0,3.0,4.0
u5,NaN,3.0,5.0


Cosine similarity (b, c): -0.951
Cosine similarity (b, f): 1.0

 Cụm dữ liệu có chứa d :


,a,d,e
u1,5.0,4.0,3.0
u2,4.0,NaN,5.0
u3,NaN,1.0,1.0
u4,7.0,6.0,NaN
u5,1.0,2.0,2.0


Cosine similarity (d, a): -0.604
Cosine similarity (d, e): 0.316


,item,pred_score
1,d,5.0
0,b,4.0


### Ví dụ 2:
Áp dụng thuật toán k-means + lọc cộng tác dựa trên user cho tập dữ liệu data01.xls với userID=89

In [11]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

# Thay đổi đương dẫn
import os
os.chdir("drive/My Drive/KHTN/RecSys/data")

# Print out the current directory
!pwd

Mounted at /content/drive
/content/drive/My Drive/KHTN/RecSys/data


In [12]:
# Load the dataset
data = pd.read_excel('data_01.xls')
data.rename(columns={"Unnamed: 0": "userId"}, inplace=True)
data.set_index('userId', inplace = True)
data.head()

,11: Star Wars: Episode IV - A New Hope (1977),12: Finding Nemo (2003),13: Forrest Gump (1994),14: American Beauty (1999),22: Pirates of the Caribbean: The Curse of the Black Pearl (2003),24: Kill Bill: Vol. 1 (2003),38: Eternal Sunshine of the Spotless Mind (2004),63: Twelve Monkeys (a.k.a. 12 Monkeys) (1995),77: Memento (2000),85: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),...,8467: Dumb & Dumber (1994),8587: The Lion King (1994),9331: Clear and Present Danger (1994),9741: Unbreakable (2000),9802: The Rock (1996),9806: The Incredibles (2004),10020: Beauty and the Beast (1991),36657: X-Men (2000),36658: X2: X-Men United (2003),36955: True Lies (1994)
userId,,,,,,,,,,,,,,,,,,,,,
1648,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,5.0,3.5,3.0,NaN,3.5,NaN
5136,4.5,5.0,5.0,4.0,5.0,5.0,5.0,3.0,NaN,5.0,...,1.0,5.0,NaN,NaN,NaN,5.0,5.0,4.5,4.0,NaN
918,5.0,5.0,4.5,NaN,3.0,NaN,5.0,NaN,5.0,NaN,...,NaN,5.0,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN
2824,4.5,NaN,5.0,NaN,4.5,4.0,NaN,NaN,5.0,NaN,...,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3867,4.0,4.0,4.5,NaN,4.0,3.0,NaN,NaN,NaN,4.5,...,1.0,4.0,NaN,NaN,NaN,3.0,4.0,4.0,3.5,3.0


In [22]:
userID = 89
n_cluster = 3
i_cluster, cens = k_means(data, n_cluster)

i_cluster_df = pd.DataFrame(i_cluster.items(), columns=['item', 'cluster'])

for i in range(n_cluster):
  n_items = i_cluster_df[i_cluster_df['cluster'] == i].shape[0]
  print(f'Số lượng item trong cluster {i}: {n_items}')

item_based_rec(data, i_cluster_df, userID).head(10)

Số lượng item trong cluster 0: 13
Số lượng item trong cluster 1: 75
Số lượng item trong cluster 2: 12


,item,pred_score
33,807: Seven (a.k.a. Se7en) (1995),5.000000
39,1422: The Departed (2006),4.734713
23,568: Apollo 13 (1995),4.733541
17,274: The Silence of the Lambs (1991),4.707737
30,745: The Sixth Sense (1999),4.697952
5,98: Gladiator (2000),4.630701
52,8587: The Lion King (1994),4.581676
27,640: Catch Me If You Can (2002),4.572580
4,85: Raiders of the Lost Ark (Indiana Jones and...,4.549002
25,597: Titanic (1997),4.518342


### **Bài tập:**

Sử dụng tập dữ liệu trong ví dụ 2 bên trên (`data01.xls`)

**Yêu cầu**:
- Áp dụng k-means để phân thành 3 cụm user;
- Sử dụng thuật toán lọc cộng tác dựa trên user để tiến hành dự đoán rating của các item mà user 89 chưa đánh giá.